In [ ]:
import numpy as np
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup
from urllib.request import Request
from urllib.request import urlopen

from tqdm import tqdm_notebook

import time

In [ ]:
driver = webdriver.Chrome('chromedriver.exe')
driver.get('https://www.ediya.com/contents/find_store.html#c')

# Wait two seconds and click the button "[주소]"
time.sleep(2)
driver.find_element(By.XPATH, '//*[@id="contentWrap"]/div[3]/div/div[1]/ul/li[2]/a').click()

In [ ]:
gu_names = ['강남구','강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구', '동대문구', '동작구',
 '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구', '영등포구', '용산구', '은평구', '종로구', '중구',
 '중랑구'] 

# list(starbucks['Area'].unique())

In [ ]:
# Variables

search_box = driver.find_element(By.XPATH, '//*[@id="keyword"]')
search_btn = driver.find_element(By.XPATH, '//*[@id="keyword_div"]/form/button')

In [ ]:
# Function to get detailed (old) address from an overlay box

def GetDetailAddr(br):
    br.click()
    time.sleep(1)

    go_details = driver.find_element(By.CLASS_NAME, 'st_info_btn').find_element(By.TAG_NAME, 'img')
    go_details.click()
    time.sleep(1)

    addr_detail = driver.find_element(By.TAG_NAME, 'dd')
    target = addr_detail.text
    time.sleep(1)

    close = driver.find_element(By.CLASS_NAME, 'pop_close').find_element(By.TAG_NAME,'a')
    close.click()
    time.sleep(1)

    return target

In [ ]:
# Function to generate empty dataframe

def GenerateDataFrame(Area = [], Branch = [], Addr = []):
    tmp = (
        {'Area': Area,
        'Branch' : Branch,
        'Address' : Addr,
        }
    )
    return pd.DataFrame(tmp)

ediya = GenerateDataFrame()
ediya.head()

In [ ]:
for gu in tqdm_notebook(gu_names): 
    search_box.clear()
    search_box.send_keys('서울 ' + gu) # In case there are multiple cities having the same district name. (ex: 부산 강서구, 인천 중구)
    time.sleep(1)

    search_btn.click()
    time.sleep(1)

    places = driver.find_elements(By.CLASS_NAME, 'item')
    area = []; branch = []; address = []; 
    for p in places:
        tmp = p.text.split('\n')
        branch.append(tmp[0])

        if tmp[1][-1] == '동': # Some branches provide old address in an overlay box. Get information from it.
            address.append(GetDetailAddr(p))
        else: 
            address.append(tmp[1])

        area = [gu for i in range(len(address))]

    ediya_tmp = GenerateDataFrame(area, branch, address)    
    ediya = pd.concat([ediya, ediya_tmp])

    time.sleep(1)

ediya.head()

In [ ]:
ediya.info()

In [ ]:
# ediya.reset_index(inplace=True)
ediya.set_index('index', inplace=True)
ediya.info()

In [ ]:
ediya.head()

In [ ]:
uniq = ediya['Area'].unique()
print(len(uniq), '\n', uniq)

In [ ]:
ediya.to_csv('data/01_ediya.csv', sep =',', encoding= 'cp949')

In [ ]:
driver.close()